In [ ]:
% matplotlib inline
import matplotlib.pyplot as plt

import tensorflow as tf

import sys
sys.path.insert(0, '/home/jr0th/github/segmentation/code/')
import helper.model_builder
import helper.metrics

import skimage.io

import numpy as np

import time

import keras

# build session running on a specific GPU
configuration = tf.ConfigProto()
configuration.gpu_options.allow_growth = True
configuration.gpu_options.visible_device_list = "2"
session = tf.Session(config = configuration)

keras.backend.set_session(session)

In [ ]:
img_load_pattern = '/home/jr0th/github/segmentation/data/BBBC022_hand_200/test/x/all/*.png'

# uncomment the following line to use full images instead of crops
img_load_pattern = '/home/jr0th/github/segmentation/data/BBBC022_hand_200/test/x_big/*.png'

# use latest checkpoint
weights_path = '/home/jr0th/github/segmentation/checkpoints/checkpoint_boundary_4_generator.hdf5'

# get images
images = skimage.io.imread_collection(img_load_pattern).concatenate()

# assume that images are all the same shape
n_images = images.shape[0]
dim1 = images.shape[1]
dim2 = images.shape[2]

# build model and load weights
model = helper.model_builder.get_model_1_class(dim1, dim2)

model.load_weights(weights_path)

# TEST INFERENCE AND POST PROCESSING TIME

In [ ]:
# label = 'PATCH'
# label = 'PATCH_thresh'
# label = 'PATCH_remove_small'
# label = 'PATCH_thresh_remove_small'

# label = 'FULL'
# label = 'FULL_thresh'
# label = 'FULL_remove_small'
label = 'FULL_thresh_remove_small'

N = 100

time_inf = np.zeros(N)
time_pp = np.zeros(N)

batch_size = 10

print('Heating up GPU now.')

# heat up GPU
for i in range(10):
    model.predict(np.zeros((batch_size, dim1, dim2, 1)), batch_size=batch_size)

print('GPU is hot now.')

for i in range(N):
    
    print('Iteration', i+1, 'of', N)
    
    # get random image
    img = images[np.random.randint(0, n_images)]

    # preprocessing
    img_rescaled = np.reshape(img / 255, (-1, dim1, dim2, 1))

    # inference
    start_inf = time.time()
    outline_pred = model.predict(img_rescaled)
    end_inf = time.time()

    time_inf[i] = end_inf - start_inf

    # postprocessing
    outline_pred = outline_pred.squeeze()
    start_pp = time.time()
    contour = helper.metrics.probmap_to_contour(outline_pred)
    segmentation = helper.metrics.contour_to_label(contour, img_rescaled)
    end_pp = time.time()

    time_pp[i] = end_pp - start_pp
    
    # visualize
    visualize = False
    if(visualize):
        plt.imshow(img)
        plt.colorbar()
        plt.title('Image')
        plt.show()

        plt.imshow(outline_pred)
        plt.colorbar()
        plt.title('Outline Probability Map')
        plt.show()

        plt.imshow(contour)
        plt.colorbar()
        plt.title('Thresholded Contours')
        plt.show()

        plt.imshow(segmentation)
        plt.colorbar()
        plt.title('Predicted Segmentation')
        plt.show()

plt.boxplot([time_inf[1:], time_pp[1:]], labels=['Forward Pass', 'Post Processing'])
plt.title('Running Time')
plt.ylabel('time in seconds')
plt.savefig('inf_pp_time_' + label + '.svg', format='svg')

np.save('inf_pp_time_' + label + '.npy',[time_inf, time_pp])

print('Mean time for inference: ', np.mean(time_inf))
print('Mean time for inference: ', np.mean(time_pp))

# TEST DEPENDENCY ON BATCH SIZE

In [ ]:
max_n_images = 15
batch_size = 5

N = 100

runtimes_per_image = np.zeros((N, max_n_images))

print('Heating up GPU now.')

# heat up GPU
for i in range(10):
    model.predict(np.zeros((batch_size, dim1, dim2, 1)), batch_size=batch_size)

print('GPU is hot now.')

# loop over iterations
for i in range(N):
    
    # loop over number of images
    for n_images in range(1, max_n_images + 1):

        # prepare data
        data = np.zeros((n_images, dim1, dim2, 1))
        for j in range(n_images):
            # get random image
            data[j,:,:,0] = images[j]

        # preprocessing
        data = data / 255

        # inference
        time_inf = 0
        start_inf = time.time()
        outline_pred = model.predict(data, batch_size=batch_size)
        end_inf = time.time()

        time_inf = end_inf - start_inf

        # visualize
        visualize = False
        if(visualize):
            plt.imshow(outline_pred[0].squeeze())
            plt.show()

        runtimes_per_image[i,n_images-1] = time_inf / n_images
        print('Time for inference per image:', np.round(runtimes_per_image[i, n_images-1], decimals=2), 'with', n_images, 'samples')

In [ ]:
plt.boxplot(runtimes_per_image[1:])
plt.title('Running Time')
plt.ylabel('inference time per image in seconds')
plt.xlabel('data set size (batch size = '+str(batch_size)+')')
plt.savefig('runtime_batch_size.svg', format='svg')

np.save('runtimes_per_image.npy', runtimes_per_image)

# HELPER TO GET MEAN VALUES

In [ ]:
data = np.load('./inf_pp_time_PATCH_thresh_remove_small.npy')
np.mean(data[:,1:], axis=1)